In [11]:
# doing some imports
import json
import pickle
from haystack.document_store.elasticsearch import ElasticsearchDocumentStore

In [2]:
# bringing in the list of the answers and the 
# the questions and the context from the squad list.
#
with open("pred_ans_list", mode="rb") as f:
    pred_ans_list = pickle.load(f,)


with open("true_ans_list", mode="rb") as f:
    true_ans_list = pickle.load(f)

with open("questions_list", mode="rb") as f:
    questions_list = pickle.load(f)

with open("contexts", mode="rb") as f:
    contexts = pickle.load(f)

In [3]:
print(len(pred_ans_list))
pred_ans_list[:3]

print(f"The length of the contexts that is brought in is {len(contexts)}")

10000
The length of the contexts that is brought in is 130319


In [4]:
print(len(true_ans_list))
true_ans_list[:3]

10000


['in the late 1990s', 'singing and dancing', '2003']

In [5]:
with open("squad_ans", mode="rb") as f:
    squad_ans = pickle.load(f)

In [16]:
e = ElasticsearchDocumentStore(
    host = "localhost", username="", 
    password="", index="squad_docs"
)


/home/rich/anaconda3/envs/ml/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
11/22/2021 17:45:34 - INFO - elasticsearch -   HEAD http://localhost:9200/ [status:200 request:0.037s]
11/22/2021 17:45:34 - INFO - elasticsearch -   HEAD http://localhost:9200/squad_docs [status:200 request:0.007s]
11/22/2021 17:45:34 - INFO - elasticsearch -   GET http://localhost:9200/squad_docs [status:200 request:0.005s]
11/22/2021 17:45:34 - INFO - elasticsearch -   PUT http://localhost:9200/squad_docs/_mapping [status:200 request:0.025s]
11/22/2021 17:45:34 - INFO - elasticsearch -   HEAD http://localhost:9200/label [status:200 request:0.005s]


In [7]:
# want to see if the host is found
import requests

In [8]:
res = requests.get("http://localhost:9200/_cluster/health")

In [9]:
res.json()

{'cluster_name': 'elasticsearch',
 'status': 'green',
 'timed_out': False,
 'number_of_nodes': 1,
 'number_of_data_nodes': 1,
 'active_primary_shards': 1,
 'active_shards': 1,
 'relocating_shards': 0,
 'initializing_shards': 0,
 'unassigned_shards': 0,
 'delayed_unassigned_shards': 0,
 'number_of_pending_tasks': 0,
 'number_of_in_flight_fetch': 0,
 'task_max_waiting_in_queue_millis': 0,
 'active_shards_percent_as_number': 100.0}

In [10]:
# the data for the elastic search 
# which is as follows
# as a dictionary 
# { "text": (here is the context of the squad data)
# }
#
the_context_data = []
for each_context in contexts[:10000]:
    the_context_data.append({"text": each_context})
# looking at what they look like
the_context_data[:2]

[{'text': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".'},
 {'text': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the l

In [13]:
e.write_documents(the_context_data)

/home/rich/anaconda3/envs/ml/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
11/22/2021 17:39:14 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:2.161s]
11/22/2021 17:39:15 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.034s]
11/22/2021 17:39:16 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.098s]
11/22/2021 17:39:17 - INFO - elasticsearch -   POST http://localhost:9200/_bulk?refresh=wait_for [status:200 request:1.104s]
11/22/2021 17:39:18 - INFO - elasticsearch -   POST http://localho

Querying the data (GETTING THE DATA)

In [14]:
from haystack.retriever.sparse import TfidfRetriever

In [17]:
retriever = TfidfRetriever(e)

/home/rich/anaconda3/envs/ml/lib/python3.8/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.15/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
11/22/2021 17:46:10 - INFO - elasticsearch -   POST http://localhost:9200/squad_docs/_search?scroll=1d&size=10000 [status:200 request:1.176s]
11/22/2021 17:46:10 - INFO - elasticsearch -   POST http://localhost:9200/_search/scroll [status:200 request:0.047s]
11/22/2021 17:46:10 - INFO - elasticsearch -   DELETE http://localhost:9200/_search/scroll [status:200 request:0.027s]
11/22/2021 17:46:10 - INFO - haystack.retriever.sparse -   Found 1593 candidate paragraphs from 1593 docs in DB


In [18]:
len(the_context_data)

10000